# Residual Networks

Welcome to the second assignment of this week! You will learn how to build very deep convolutional networks, using Residual Networks (ResNets). In theory, very deep networks can represent very complex functions; but in practice, they are hard to train. Residual Networks, introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf), allow you to train much deeper networks than were previously practically feasible.

**In this assignment, you will:**
- Implement the basic building blocks of ResNets. 
- Put together these building blocks to implement and train a state-of-the-art neural network for image classification. 

This assignment will be done in Keras. 

Before jumping into the problem, let's run the cell below to load the required packages.

In [1]:
import numpy as np
from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
import torch
from torch import nn
from torch.nn import functional
import math
from resnets_utils import load_dataset
%matplotlib inline

/home/tyler/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [103]:
class IdentityBlock(nn.Module):
    def __init__(self, in_channel, out_channels, f, in_size, conv_block=False,
                s=2):
        super(IdentityBlock, self).__init__()
        if conv_block:
            self.conv1 = nn.Conv2d(in_channel, out_channels[0], 1, s)
            out1_size = int(((in_size - 1) / s) + 1)
        else:
            self.conv1 = nn.Conv2d(in_channel, out_channels[0], 1)
            out1_size = int(((in_size - 1) / 1) + 1)
        self.bn1 = nn.BatchNorm2d(out_channels[0])
        ## we want same padding here
        conv2_padding = int((f-1)/2)
        self.conv2 = nn.Conv2d(out_channels[0], out_channels[1], f, 
                               padding=conv2_padding)
        self.bn2 = nn.BatchNorm2d(out_channels[1])
        self.conv3 = nn.Conv2d(out_channels[1], out_channels[2], 1)
        self.bn3 = nn.BatchNorm2d(out_channels[2])
        
        #if conv block need some additional layers
        self.conv_block = conv_block
        if self.conv_block:
            self.shortcut_conv = nn.Conv2d(in_channel, out_channels[2], 1, stride=s)
            self.shortcut_bn = nn.BatchNorm2d(out_channels[2])
        
    def forward(self, x):
        conv1_out = functional.relu(self.bn1(self.conv1(x)))
        conv2_out = functional.relu(self.bn2(self.conv2(conv1_out)))
        conv3_out = self.bn3(self.conv3(conv2_out))
        if self.conv_block:
            shortcut = self.shortcut_bn(self.shortcut_conv(x))
        else:
            shortcut = x
        return functional.relu(shortcut + conv3_out)
    
class ResNet50(nn.Module):
    def __init__(self, classes):
        super(ResNet50, self).__init__()
        
        #stage 1
        self.conv1 = nn.Conv2d(3, 64, 7, 2, 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.mp1 = nn.MaxPool2d(3, stride=2)
        
        #stage 2
        self.conv_block1 = IdentityBlock(64, [64,64,256], 3, 15,
                                         conv_block=True, s=1)
        self.id_block1 = IdentityBlock(256, [64,64,256], 3, 15)
        self.id_block2 = IdentityBlock(256, [64,64,256], 3, 15)
        
        #stage 3
        self.conv_block2 = IdentityBlock(256, [128,128,512], 3, 15,
                                         conv_block=True, s=2)
        self.id_block3 = IdentityBlock(512, [128,128,512], 3, 15)
        self.id_block4 = IdentityBlock(512, [128,128,512], 3, 15)
        self.id_block5 = IdentityBlock(512, [128,128,512], 3, 15)
        
        #stage 4
        self.conv_block3 = IdentityBlock(512, [256, 256, 1024], 3, 8,
                                         conv_block=True, s=2)
        self.id_block6 = IdentityBlock(1024, [256, 256, 1024], 3, 8)
        self.id_block7 = IdentityBlock(1024, [256, 256, 1024], 3, 8)
        self.id_block8 = IdentityBlock(1024, [256, 256, 1024], 3, 8)
        self.id_block9 = IdentityBlock(1024, [256, 256, 1024], 3, 8)
        self.id_block10 = IdentityBlock(1024, [256, 256, 1024], 3, 8)
        
        #stage 5
        self.conv_block4 = IdentityBlock(1024, [512, 512, 2048], 3, 4,
                                         conv_block=True, s=2)
        self.id_block11 = IdentityBlock(2048, [512, 512, 2048], 3, 4)
        self.id_block12 = IdentityBlock(2048, [512, 512, 2048], 3, 4)
        
        #stage 6
        self.ap = nn.AvgPool2d(2)
        self.linear = nn.Linear(2048, classes)
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.mp1(functional.relu(self.bn1(self.conv1(x))))
        x = self.conv_block1(x)
        x = self.id_block2(self.id_block1(x))
        x = self.id_block5(self.id_block4(self.id_block3(self.conv_block2(x))))
        x = self.id_block8(self.id_block7(self.id_block6(self.conv_block3(x))))
        x = self.id_block10(self.id_block9(x))
        x = self.id_block12(self.id_block11(self.conv_block4(x)))
        x = self.ap(x).view(batch_size, -1)
        x = self.linear(x)
        return x

In [104]:
X_train_orig, Y_train, X_test_orig, Y_test, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1, 1080)
X_test shape: (120, 64, 64, 3)
Y_test shape: (1, 120)


In [105]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

resnet = ResNet50(6).to(device)

In [106]:
data_in = torch.FloatTensor(X_train[0]).view(1,3,64,64).to(device)

resnet(data_in)

tensor([[-0.1661,  0.3251,  0.0765,  0.3080, -0.1592,  0.1753]], device='cuda:0')

In [ ]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

**Expected Output**:

<table>
    <tr>
        <td>
            ** Epoch 1/2**
        </td>
        <td>
           loss: between 1 and 5, acc: between 0.2 and 0.5, although your results can be different from ours.
        </td>
    </tr>
    <tr>
        <td>
            ** Epoch 2/2**
        </td>
        <td>
           loss: between 1 and 5, acc: between 0.2 and 0.5, you should see your loss decreasing and the accuracy increasing.
        </td>
    </tr>

</table>

Let's see how this model (trained on only two epochs) performs on the test set.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**Expected Output**:

<table>
    <tr>
        <td>
            **Test Accuracy**
        </td>
        <td>
           between 0.16 and 0.25
        </td>
    </tr>

</table>

For the purpose of this assignment, we've asked you to train the model only for two epochs. You can see that it achieves poor performances. Please go ahead and submit your assignment; to check correctness, the online grader will run your code only for a small number of epochs as well.

After you have finished this official (graded) part of this assignment, you can also optionally train the ResNet for more iterations, if you want. We get a lot better performance when we train for ~20 epochs, but this will take more than an hour when training on a CPU. 

Using a GPU, we've trained our own ResNet50 model's weights on the SIGNS dataset. You can load and run our trained model on the test set in the cells below. It may take ≈1min to load the model.

In [ ]:
model = load_model('ResNet50.h5') 

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

ResNet50 is a powerful model for image classification when it is trained for an adequate number of iterations. We hope you can use what you've learnt and apply it to your own classification problem to perform state-of-the-art accuracy.

Congratulations on finishing this assignment! You've now implemented a state-of-the-art image classification system! 

## 4 - Test on your own image (Optional/Ungraded)

If you wish, you can also take a picture of your own hand and see the output of the model. To do this:
    1. Click on "File" in the upper bar of this notebook, then click "Open" to go on your Coursera Hub.
    2. Add your image to this Jupyter Notebook's directory, in the "images" folder
    3. Write your image's name in the following code
    4. Run the code and check if the algorithm is right! 

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
my_image = scipy.misc.imread(img_path)
imshow(my_image)
print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(model.predict(x))

You can also print a summary of your model by running the following code.

In [ ]:
model.summary()

Finally, run the code below to visualize your ResNet50. You can also download a .png picture of your model by going to "File -> Open...-> model.png".

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

<font color='blue'>
**What you should remember:**
- Very deep "plain" networks don't work in practice because they are hard to train due to vanishing gradients.  
- The skip-connections help to address the Vanishing Gradient problem. They also make it easy for a ResNet block to learn an identity function. 
- There are two main type of blocks: The identity block and the convolutional block. 
- Very deep Residual Networks are built by stacking these blocks together.

### References 

This notebook presents the ResNet algorithm due to He et al. (2015). The implementation here also took significant inspiration and follows the structure given in the github repository of Francois Chollet: 

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's github repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
